In [1]:
from google.oauth2 import service_account

# credentials.json dosyasının yolu
credentials_path =  "tough-history5d2.json"
credentials = service_account.Credentials.from_service_account_file(credentials_path)

In [2]:
PROJECT_ID = 'tough-history-431913'
REGION = 'us-central1'

In [3]:
import vertexai
vertexai.init(project=PROJECT_ID, location=REGION, credentials = credentials)

In [39]:
embedding_model = TextEmbeddingModel.from_pretrained(
    "textembedding-gecko@001")

In [16]:
with open('/home/text1.txt', 'r') as file:
    text = file.read()

In [17]:
news = text.strip().split("\n\n")


In [18]:
for i in news:
    print(i)
    print("..")

İlk Türk astronot Gezeravcı'nın da yer aldığı Ax-3 ekibi Uluslararası Uzay İstasyonu'nda
..
Türk, İspanyol-Amerikan, İtalyan ve İsveçli olmak üzere çok uluslu Ax-3 ekibini taşıyan Dragon kapsülünün alçak dünya yörüngesindeki laboratuvar olarak tanımlanan UUİ'ye kenetlenme anı, Axiom Space'in internet sitesinden canlı yayınlandı.
Yaklaşık 36 saat süren uzay yolculuğunun ardından ilk Türk astronot Gezeravcı'nın da yer aldığı Ax-3 ekibi, ABD yerel saatiyle sabah 05.42 (TSİ 13.42) civarında UUİ'ye ulaşt
UUİ'de bulunan, misyonları 27 Eylül 2023'te başlayan Expedition 70 astronot ve kozmonotları , Ax-3 ekibini, istasyonda karşıladı
Gezeravcı, 2 hafta boyunca uzayda bilimsel çalışmalar yapacak
Gezeravcı, 14 gün boyunca kalacağı UUİ'de 13 farklı bilimsel deney üzerinde çalışacak.
Bu deneyler, mikro yer çekimi, uzay ortamında insan sağlığı, Tuz Gölü bitkisinin uzay ortamında araştırılması, katı-akışkan karışımların yerçekimsiz ortamda araştırılması gibi alanlarda çeşitlilik gösteriyor.
..
Zeyne

In [25]:
data=[]

for i in range(0,len(news),3):
    headline = news[i].strip()
    content = news[i+1].strip()
    category_author_date = news[i+2].strip()

    category_author = category_author_date.strip().split("\n")
    author = category_author[0].strip()
    cat_date = category_author[1].split(' ')
    date = cat_date[1].strip()
    category = cat_date[0].strip()

    data.append([headline,content,category,author,date])

    

In [30]:
import time
from concurrent.futures import ThreadPoolExecutor
from tqdm.auto import tqdm
import functools
from vertexai.language_models import TextEmbeddingModel
import math
import numpy as np


def generate_batches(sentences, batch_size = 5):
    for i in range(0, len(sentences), batch_size):
        yield sentences[i : i + batch_size]

def encode_texts_to_embeddings(sentences):
    model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")
    try:
        embeddings = model.get_embeddings(sentences)
        return [embedding.values for embedding in embeddings]
    except Exception:
        return [None for _ in range(len(sentences))]
    
def encode_text_to_embedding_batched(sentences, api_calls_per_second = 0.33, batch_size = 5):
    # Generates batches and calls embedding API
    
    embeddings_list = []

    # Prepare the batches using a generator
    batches = generate_batches(sentences, batch_size)

    seconds_per_job = 1 / api_calls_per_second

    with ThreadPoolExecutor() as executor:
        futures = []
        for batch in tqdm(
            batches, total = math.ceil(len(sentences) / batch_size), position=0
        ):
            futures.append(
                executor.submit(functools.partial(encode_texts_to_embeddings), batch)
            )
            time.sleep(seconds_per_job)

        for future in futures:
            embeddings_list.extend(future.result())

    is_successful = [
        embedding is not None for sentence, embedding in zip(sentences, embeddings_list)
    ]
    embeddings_list_successful = np.squeeze(
        np.stack([embedding for embedding in embeddings_list if embedding is not None])
    )
    return embeddings_list_successful

In [26]:
import pandas as pd


In [29]:
news_data = pd.DataFrame(data, columns=['title', 'content', 'category', 'author', 'date'])
news_data.head()

,title,content,category,author,date
0,İlk Türk astronot Gezeravcı'nın da yer aldığı ...,"Türk, İspanyol-Amerikan, İtalyan ve İsveçli ol...",Gündem-Dünya,"Zeynep Katre Oran, Irmak Akcan",2024.01.20
1,Ordu'nun Fatsa ilçesinde şiddetli yağış su bas...,Ordu'nun Fatsa ilçesinde etkili olan şiddetli ...,Gündem,Hayati Akçay,08.07.2024


In [46]:
news_list = news_data.content.tolist()
news_embeddings = encode_text_to_embedding_batched(
            sentences = news_list,
            api_calls_per_second = 20/60, 
            batch_size = 5)

  0%|          | 0/1 [00:00<?, ?it/s]

In [48]:
# import pickle

# with open('news_embeddings.pkl', 'wb') as file:
#     pickle.dump(news_embeddings, file)

In [49]:
import pickle
with open('news_embeddings.pkl', 'rb') as file:
      
    # Call load method to deserialze
    news_embeddings = pickle.load(file)
  
    print(news_embeddings)

[[-0.06223642  0.00290349  0.00538724 ...  0.00327592 -0.02683627
  -0.00044584]
 [-0.0238795   0.00681837 -0.01852706 ... -0.0016953  -0.00509094
  -0.01097084]]


In [50]:
news_data["embeddings"] = news_embeddings.tolist()
news_data.head()

,title,content,category,author,date,embeddings
0,İlk Türk astronot Gezeravcı'nın da yer aldığı ...,"Türk, İspanyol-Amerikan, İtalyan ve İsveçli ol...",Gündem-Dünya,"Zeynep Katre Oran, Irmak Akcan",2024.01.20,"[-0.06223641708493233, 0.002903488464653492, 0..."
1,Ordu'nun Fatsa ilçesinde şiddetli yağış su bas...,Ordu'nun Fatsa ilçesinde etkili olan şiddetli ...,Gündem,Hayati Akçay,08.07.2024,"[-0.023879501968622208, 0.0068183704279363155,..."


##### Semantic search

In [51]:
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances_argmin as distances_argmin

In [76]:
query = ['space']
query_embedding = embedding_model.get_embeddings(query)[0].values

In [78]:
cos_sim_array = cosine_similarity([query_embedding],
                                  list(news_data.embeddings.values))

In [79]:
cos_sim_array.shape

(1, 2)

In [80]:
index_doc_cosine = np.argmax(cos_sim_array)

In [81]:
index_doc_distances = distances_argmin([query_embedding], 
                                       list(news_data.embeddings.values))[0]

In [82]:
news_data.content[index_doc_cosine]

"Türk, İspanyol-Amerikan, İtalyan ve İsveçli olmak üzere çok uluslu Ax-3 ekibini taşıyan Dragon kapsülünün alçak dünya yörüngesindeki laboratuvar olarak tanımlanan UUİ'ye kenetlenme anı, Axiom Space'in internet sitesinden canlı yayınlandı.\nYaklaşık 36 saat süren uzay yolculuğunun ardından ilk Türk astronot Gezeravcı'nın da yer aldığı Ax-3 ekibi, ABD yerel saatiyle sabah 05.42 (TSİ 13.42) civarında UUİ'ye ulaşt\nUUİ'de bulunan, misyonları 27 Eylül 2023'te başlayan Expedition 70 astronot ve kozmonotları , Ax-3 ekibini, istasyonda karşıladı\nGezeravcı, 2 hafta boyunca uzayda bilimsel çalışmalar yapacak\nGezeravcı, 14 gün boyunca kalacağı UUİ'de 13 farklı bilimsel deney üzerinde çalışacak.\nBu deneyler, mikro yer çekimi, uzay ortamında insan sağlığı, Tuz Gölü bitkisinin uzay ortamında araştırılması, katı-akışkan karışımların yerçekimsiz ortamda araştırılması gibi alanlarda çeşitlilik gösteriyor."

In [83]:
news_data.content[index_doc_distances]

"Türk, İspanyol-Amerikan, İtalyan ve İsveçli olmak üzere çok uluslu Ax-3 ekibini taşıyan Dragon kapsülünün alçak dünya yörüngesindeki laboratuvar olarak tanımlanan UUİ'ye kenetlenme anı, Axiom Space'in internet sitesinden canlı yayınlandı.\nYaklaşık 36 saat süren uzay yolculuğunun ardından ilk Türk astronot Gezeravcı'nın da yer aldığı Ax-3 ekibi, ABD yerel saatiyle sabah 05.42 (TSİ 13.42) civarında UUİ'ye ulaşt\nUUİ'de bulunan, misyonları 27 Eylül 2023'te başlayan Expedition 70 astronot ve kozmonotları , Ax-3 ekibini, istasyonda karşıladı\nGezeravcı, 2 hafta boyunca uzayda bilimsel çalışmalar yapacak\nGezeravcı, 14 gün boyunca kalacağı UUİ'de 13 farklı bilimsel deney üzerinde çalışacak.\nBu deneyler, mikro yer çekimi, uzay ortamında insan sağlığı, Tuz Gölü bitkisinin uzay ortamında araştırılması, katı-akışkan karışımların yerçekimsiz ortamda araştırılması gibi alanlarda çeşitlilik gösteriyor."

#### Question answering with relevant context


In [84]:
from vertexai.language_models import TextGenerationModel

In [85]:
generation_model = TextGenerationModel.from_pretrained(
    "text-bison@001")

In [90]:
query = 'Uzay uçusu ne zaman oldu'
query_embedding = embedding_model.get_embeddings(query)[0].values

In [91]:
context = "News: " + news_data.content[index_doc_cosine] +\
"\n Title: " + news_data.title[index_doc_cosine] +\
"\n Date:" + news_data.date[index_doc_cosine] +\
"\n Category: " + news_data.category[index_doc_cosine] 

In [94]:
prompt = f"""Here is the context: {context}
             Give me response in Turkish.
             Using the relevant information from the context,
             provide an answer to the query: {query}."
             If the context doesn't provide \
             any relevant information, \
             answer with \
             [I couldn't find a good match in the \
             document database for your query]
             """

In [95]:
from IPython.display import Markdown, display

t_value = 0.2
response = generation_model.predict(prompt = prompt,
                                    temperature = t_value,
                                    max_output_tokens = 1024)

display(Markdown(response.text))

Uzay uçuşu 20 Ocak 2024'te gerçekleşti.

In [97]:
query = ['How to make the perfect lasagna']
query_embedding = embedding_model.get_embeddings(query)[0].values

cos_sim_array = cosine_similarity([query_embedding], 
                                  list(news_data.embeddings.values))
index_doc = np.argmax(cos_sim_array)

context = "News: " + news_data.content[index_doc_cosine] +\
"\n Title: " + news_data.title[index_doc_cosine] +\
"\n Date:" + news_data.date[index_doc_cosine] +\
"\n Category: " + news_data.category[index_doc_cosine] 

prompt = f"""Here is the context: {context}
             Give me response in Turkish.
             Using the relevant information from the context,
             provide an answer to the query: {query}."
             If the context doesn't provide \
             any relevant information, \
             answer with \
             [I couldn't find a good match in the \
             document database for your query]
             """

t_value = 0.2
response = generation_model.predict(prompt = prompt,
                                    temperature = t_value,
                                    max_output_tokens = 1024)
display(Markdown(response.text))

I couldn't find a good match in the document database for your query.

In [ ]:
def create_index(embedded_dataset, 
                 num_leaves,
                 num_leaves_to_search,
                 training_sample_size):
    
    # normalize data to use cosine sim as explained in the paper
    normalized_dataset = embedded_dataset / np.linalg.norm(embedded_dataset, axis=1)[:, np.newaxis]
    
    searcher = (
        scann.scann_ops_pybind.builder(normalized_dataset, 10, "dot_product")
        .tree(
            num_leaves = num_leaves,
            num_leaves_to_search = num_leaves_to_search,
            training_sample_size = training_sample_size,
        )
        .score_ah(2, anisotropic_quantization_threshold = 0.2)
        .reorder(100)
        .build()
    )
    return searcher

## Scale with approximate nearest neighbor search


In [ ]:
import scann

#Create index using scann
index = create_index(embedded_dataset = question_embeddings, 
                     num_leaves = 25,
                     num_leaves_to_search = 10,
                     training_sample_size = 2000)

query = "how to concat dataframes pandas"

import time 

start = time.time()
query_embedding = embedding_model.get_embeddings([query])[0].values
neighbors, distances = index.search(query_embedding, final_num_neighbors = 1)
end = time.time()

for id, dist in zip(neighbors, distances):
    print(f"[docid:{id}] [{dist}] -- {so_database.input_text[int(id)][:125]}...")

print("Latency (ms):", 1000 * (end - start))

start = time.time()
query_embedding = embedding_model.get_embeddings([query])[0].values
cos_sim_array = cosine_similarity([query_embedding], list(so_database.embeddings.values))
index_doc = np.argmax(cos_sim_array)
end = time.time()

print(f"[docid:{index_doc}] [{np.max(cos_sim_array)}] -- {so_database.input_text[int(index_doc)][:125]}...")

print("Latency (ms):", 1000 * (end - start))


